In [2]:
import numpy as np
import pandas as pd
import datetime

In [3]:
np.random.seed(42)

In [21]:
#시계열 데이타: 행 인덱스가 날짜나 시간인 데이타
#행 인덱스로 사용할 DatetimeIndex 생성 : date_range()함수
pd.date_range(start='2021/1/1',end='2021/1/4')#freq디폴트는 'D'
pd.date_range(start='2021/1/1',periods=4)
pd.date_range(start='2021/1/1',end='2021/1/4',periods=10)#1/1일부터 1/4일까지 10개의 시계열 데이타 생성
pd.date_range(start='2021/1/1',periods=4,freq='M')#말일 기준
pd.date_range(start='2021/1/1',periods=4,freq='MS')#1일 기준
pd.date_range(start='2021/1/1',periods=4,freq='3MS')#3개월단위씩 1일시작으로 4개의 시계열 데이타

DatetimeIndex(['2021-01-01', '2021-04-01', '2021-07-01', '2021-10-01'], dtype='datetime64[ns]', freq='3MS')

In [24]:
#※ '2022, 1, 21' 형태의 날짜는 반드시 빈공백 한칸 이상이 있어야 한다. 
#  그렇지 않으면 ParseError
pd.date_range('2022, 01, 21',periods=5,freq='D')

DatetimeIndex(['2022-01-21', '2022-01-22', '2022-01-23', '2022-01-24',
               '2022-01-25'],
              dtype='datetime64[ns]', freq='D')

In [29]:
#DatetimeIndex를 사용해서 시계열 데이타(데이타 프레임) 생성
data =[[900,2000,1000,900],[1500,2500,900,1500],[100,2800,1500,2000],[7000,8000,75000,6000]]
np.array(data).shape
index = pd.date_range('20220121',periods=4,freq='D')
df = pd.DataFrame(data,index=index,columns=['Open','High','Low','Close'])
df

,Open,High,Low,Close
2022-01-21,900,2000,1000,900
2022-01-22,1500,2500,900,1500
2022-01-23,100,2800,1500,2000
2022-01-24,7000,8000,75000,6000


In [48]:
#시계열 데이타관련 유용한 함수 : shift()함수
#periods(디폴트 1)에 지정한 기간 만큼 데이타를 이동시킨다.빈자리는 NaN으로 채운다
df.shift()
df.shift(periods=3)
df.shift(periods=3,fill_value=df.mean().mean())#NaN을 데이타 프레임 전체의 평균으로

,Open,High,Low,Close
2022-01-21,7100,7100,7100,7100
2022-01-22,7100,7100,7100,7100
2022-01-23,7100,7100,7100,7100
2022-01-24,900,2000,1000,900


In [49]:
#시계열 데이타관련 유용한 함수 : resample()함수-DatetimeIndexResampler타입 반환

#다운-샘플링:샘플링의 수가 줄어드는 경우. 예]일을 월로 그룹핑
#일을 월로 인덱스를 리샘플링:다운 샘플링(100개의 일 데이타가 4개의 월데이타로 다운)
#1.1~1.31까지의 시계열데이타는 1월로 그룹핑
#2.1~2.28까지의 시계열데이타는 2월로 그룹핑
#..
#4.1~4.10까지의 시계열데이타는 4월로 그룹핑
#다운-샘플링의 경우에는 원래의 데이터가 그룹으로 묶이기 때문에
#집합 연산(max(),min(),sum()등)을 해서 대표값을 구해야 한다.
ts = pd.Series(np.random.randn(100),index=pd.date_range('2022-1-1',periods=100))
ts

2022-01-01    0.496714
2022-01-02   -0.138264
2022-01-03    0.647689
2022-01-04    1.523030
2022-01-05   -0.234153
                ...   
2022-04-06   -1.463515
2022-04-07    0.296120
2022-04-08    0.261055
2022-04-09    0.005113
2022-04-10   -0.234587
Freq: D, Length: 100, dtype: float64

In [51]:
ts.head(31)
ts.tail(10)

2022-04-01    0.097078
2022-04-02    0.968645
2022-04-03   -0.702053
2022-04-04   -0.327662
2022-04-05   -0.392108
2022-04-06   -1.463515
2022-04-07    0.296120
2022-04-08    0.261055
2022-04-09    0.005113
2022-04-10   -0.234587
Freq: D, dtype: float64

In [53]:
ts.resample(rule='MS').max()#월별 최대값(시작일 기준) 시리즈

2022-01-01    1.579213
2022-02-01    1.852278
2022-03-01    1.564644
2022-04-01    0.968645
Freq: MS, dtype: float64

In [54]:
ts.resample(rule='MS').ohlc()#ohlc 메서드는 구간의 시고저종(open, high, low, close)값을 구한다.(데이타 프레임)

,open,high,low,close
2022-01-01,0.496714,1.579213,-1.913280,-0.601707
2022-02-01,1.852278,1.852278,-1.959670,0.331263
2022-03-01,0.975545,1.564644,-2.619745,0.513267
2022-04-01,0.097078,0.968645,-1.463515,-0.234587


In [55]:
#업-샘플링:샘플링의 수가 늘어나는 경우. 예]월을 일로 세분화. 데이타가 늘어나서 원래 없던 데이타를 특정 값으로 채원야 한다
#1분단위를 30초단위로 리샘플링 1개의 데이타를 2개의 데이타로

In [57]:
ts=pd.Series(np.random.randn(60),index=pd.date_range('2022-1-1',periods=60,freq='min'))
ts

2022-01-01 00:00:00   -0.974682
2022-01-01 00:01:00    0.787085
2022-01-01 00:02:00    1.158596
2022-01-01 00:03:00   -0.820682
2022-01-01 00:04:00    0.963376
2022-01-01 00:05:00    0.412781
2022-01-01 00:06:00    0.822060
2022-01-01 00:07:00    1.896793
2022-01-01 00:08:00   -0.245388
2022-01-01 00:09:00   -0.753736
2022-01-01 00:10:00   -0.889514
2022-01-01 00:11:00   -0.815810
2022-01-01 00:12:00   -0.077102
2022-01-01 00:13:00    0.341152
2022-01-01 00:14:00    0.276691
2022-01-01 00:15:00    0.827183
2022-01-01 00:16:00    0.013002
2022-01-01 00:17:00    1.453534
2022-01-01 00:18:00   -0.264657
2022-01-01 00:19:00    2.720169
2022-01-01 00:20:00    0.625667
2022-01-01 00:21:00   -0.857158
2022-01-01 00:22:00   -1.070892
2022-01-01 00:23:00    0.482472
2022-01-01 00:24:00   -0.223463
2022-01-01 00:25:00    0.714000
2022-01-01 00:26:00    0.473238
2022-01-01 00:27:00   -0.072829
2022-01-01 00:28:00   -0.846794
2022-01-01 00:29:00   -1.514847
2022-01-01 00:30:00   -0.446515
2022-01-

In [62]:
'''
업-샘플링의 경우에는 실제로 존재하지 않는 데이터를 만들어야 한다. 
이 때는 앞에서 나온 데이터(과거 데이타)를 뒤에서 그대로 쓰는 forward filling 방식(ffill()함수)과 
뒤에서 나올 데이터(최른 데이타)를 앞에서 미리 쓰는 backward filling 방식(bfill())을 사용할 수 있다.
'''
ts.resample('30s').ffill().head(10)#이전 시점의 데이타(앞에 데이타)를 새로 추가된 시점의 데이타로 사용
ts.resample('30s').bfill().head(10)#이후 데이타(뒤에 데이타)를 새로 추가된 시점의 데이타로 사용

2022-01-01 00:00:00   -0.974682
2022-01-01 00:00:30    0.787085
2022-01-01 00:01:00    0.787085
2022-01-01 00:01:30    1.158596
2022-01-01 00:02:00    1.158596
2022-01-01 00:02:30   -0.820682
2022-01-01 00:03:00   -0.820682
2022-01-01 00:03:30    0.963376
2022-01-01 00:04:00    0.963376
2022-01-01 00:04:30    0.412781
Freq: 30S, dtype: float64